# JusticeAI Quick Start Tutorial

Welcome to JusticeAI! In this tutorial, you'll learn how to:

1. Train a simple ML model
2. Evaluate its fairness with one line of code
3. Interpret the results
4. Generate interactive reports

**Time**: ~10 minutes  
**Prerequisites**: Basic Python and scikit-learn knowledge

## Setup

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Import JusticeAI
from justiceai import audit, FairnessEvaluator

print("✓ All imports successful!")

## 1. Create Synthetic Dataset

We'll create a synthetic dataset that simulates a loan approval scenario with potential gender bias.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Create features and target
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=8,
    n_redundant=2,
    n_classes=2,
    weights=[0.6, 0.4],  # Imbalanced classes
    flip_y=0.1,  # Some noise
    random_state=42
)

# Create a DataFrame
feature_names = [f'feature_{i}' for i in range(10)]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

# Add sensitive attribute: gender
# Introduce bias: positive outcomes more common for one gender
gender = np.random.choice(['Male', 'Female'], size=1000, p=[0.55, 0.45])
df['gender'] = gender

# Introduce correlation between gender and outcome (simulate bias)
male_mask = df['gender'] == 'Male'
# Make males slightly more likely to get positive outcomes
bias_flip = np.random.random(sum(male_mask)) < 0.1
df.loc[male_mask, 'target'] = np.where(
    bias_flip,
    1,
    df.loc[male_mask, 'target']
)

print(f"Dataset shape: {df.shape}")
print(f"\nTarget distribution:\n{df['target'].value_counts()}")
print(f"\nGender distribution:\n{df['gender'].value_counts()}")
print(f"\nPositive rate by gender:")
print(df.groupby('gender')['target'].mean())

## 2. Split Data and Train Model

Split the data and train a Random Forest classifier:

In [ ]:
# Separate features, target, and sensitive attribute
X = df[feature_names]
y = df['target']
gender = df['gender']

# Train/test split
X_train, X_test, y_train, y_test, gender_train, gender_test = train_test_split(
    X, y, gender, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Train model
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42
)
model.fit(X_train, y_train)

# Evaluate accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n✓ Model trained!")
print(f"Accuracy: {accuracy:.3f}")

## 3. Quick Fairness Audit

Now let's evaluate fairness with a single line of code:

In [ ]:
# One-line fairness audit
report = audit(
    model=model,
    X=X_test,
    y_true=y_test,
    sensitive_attrs=gender_test,
    fairness_threshold=0.05
)

print("✓ Fairness audit complete!")

## 4. Interpret Results

Let's examine the fairness metrics:

In [ ]:
# Get overall fairness score
score = report.get_overall_score()
print(f"Overall Fairness Score: {score:.1f}/100")

# Check if model passes fairness threshold
passes = report.passes_fairness()
print(f"Passes Fairness Threshold: {passes}")

# Get summary
summary = report.get_summary()
print(f"\nNumber of Violations: {summary['n_violations']}")
print(f"Statistical Parity Difference: {summary['statistical_parity_diff']:.4f}")
print(f"Disparate Impact Ratio: {summary['disparate_impact_ratio']:.4f}")

# Interpretation
if passes:
    print("\n✅ Model meets fairness criteria!")
else:
    print("\n⚠️ Model has fairness issues that need attention.")

## 5. Examine Fairness Issues

If there are fairness violations, let's see what they are:

In [ ]:
# Get detailed issues
issues = report.get_issues()

if len(issues) > 0:
    print(f"Found {len(issues)} fairness issue(s):\n")
    for i, issue in enumerate(issues, 1):
        print(f"{i}. {issue['metric']}")
        print(f"   {issue['message']}")
        print(f"   Severity: {issue['severity']}\n")
else:
    print("No fairness violations detected! 🎉")

## 6. Generate HTML Report

Create an interactive HTML report with visualizations:

In [ ]:
# Save HTML report
report.save_html('quickstart_fairness_report.html')
print("✓ Report saved as 'quickstart_fairness_report.html'")
print("\nOpen this file in your browser to see interactive visualizations!")

## 7. Using FairnessEvaluator for More Control

For more advanced use cases, use the `FairnessEvaluator` class:

In [ ]:
# Create evaluator with custom settings
evaluator = FairnessEvaluator(fairness_threshold=0.05)

# Evaluate model
report2 = evaluator.evaluate(
    model=model,
    X=X_test,
    y_true=y_test,
    sensitive_attrs=gender_test
)

print(f"Fairness Score: {report2.get_overall_score():.1f}/100")

## 8. Quick Check for Rapid Iteration

During model development, use `quick_check()` for fast feedback:

In [ ]:
# Quick check without generating full report
metrics = evaluator.quick_check(
    model=model,
    X=X_test,
    y_true=y_test,
    sensitive_attrs=gender_test
)

print("Quick Check Results:")
print(f"  Score: {metrics['overall_score']:.1f}")
print(f"  Passes: {metrics['passes_fairness']}")
print(f"  Violations: {metrics['n_violations']}")

## 9. Working with Pre-computed Predictions

If you already have predictions, you can evaluate them directly:

In [ ]:
# Get predictions and probabilities
predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)[:, 1]

# Evaluate predictions directly
report3 = evaluator.evaluate_predictions(
    y_true=y_test,
    y_pred=predictions,
    sensitive_attrs=gender_test,
    y_pred_proba=probabilities
)

print(f"Score from predictions: {report3.get_overall_score():.1f}/100")

## Summary

In this tutorial, you learned:

✅ How to perform a one-line fairness audit with `audit()`  
✅ How to interpret fairness scores and metrics  
✅ How to identify fairness violations  
✅ How to generate interactive HTML reports  
✅ How to use `FairnessEvaluator` for more control  
✅ How to use `quick_check()` for rapid iteration  
✅ How to evaluate pre-computed predictions  

## Next Steps

- **Tutorial 02**: Deep dive into advanced fairness metrics
- **Tutorial 03**: Customize reports and visualizations
- Check the [documentation](https://justiceai-validation.github.io/JusticeAI/) for more details

## Key Takeaways

1. **Fairness evaluation is quick**: Just one line of code!
2. **Multiple metrics matter**: No single metric tells the whole story
3. **Iterate quickly**: Use `quick_check()` during development
4. **Report for stakeholders**: Generate beautiful HTML reports for communication

Happy fair ML development! 🎯